In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import json
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from joblib import dump, load
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

In [2]:
data = pd.read_csv('feetsZTFandAPOGEE.csv')

In [3]:
counter = 0
for col in data.columns:
    print(f"[{counter}] {col}")
    counter = counter + 1

[0] oid
[1] filtercode
[2] source_id_01
[3] apogee_id_01
[4] FEETS_ra
[5] FEETS_dec
[6] FEETS_Amplitude
[7] FEETS_AndersonDarling
[8] FEETS_Autocor_length
[9] FEETS_Beyond1Std
[10] FEETS_CAR_mean
[11] FEETS_CAR_sigma
[12] FEETS_CAR_tau
[13] FEETS_Con
[14] FEETS_Eta_e
[15] FEETS_FluxPercentileRatioMid20
[16] FEETS_FluxPercentileRatioMid35
[17] FEETS_FluxPercentileRatioMid50
[18] FEETS_FluxPercentileRatioMid65
[19] FEETS_FluxPercentileRatioMid80
[20] FEETS_Freq1_harmonics_amplitude_0
[21] FEETS_Freq1_harmonics_amplitude_1
[22] FEETS_Freq1_harmonics_amplitude_2
[23] FEETS_Freq1_harmonics_amplitude_3
[24] FEETS_Freq1_harmonics_rel_phase_0
[25] FEETS_Freq1_harmonics_rel_phase_1
[26] FEETS_Freq1_harmonics_rel_phase_2
[27] FEETS_Freq1_harmonics_rel_phase_3
[28] FEETS_Freq2_harmonics_amplitude_0
[29] FEETS_Freq2_harmonics_amplitude_1
[30] FEETS_Freq2_harmonics_amplitude_2
[31] FEETS_Freq2_harmonics_amplitude_3
[32] FEETS_Freq2_harmonics_rel_phase_0
[33] FEETS_Freq2_harmonics_rel_phase_1
[34] F

In [4]:
# Meta columns to exclude
meta_columns = ['oid', 'filtercode', 'source_id_01', 'apogee_id_01']

# Columns to use for training (excluding GAIA and most APOGEE data)
training_columns = [col for col in data.columns if col.startswith('ZTF_') or col.startswith('FEETS_') or col in ['APOGEE_TEFF', 'APOGEE_FE_H','RV_TEFF', 'RV_FEH']]

# Prepare feature and target data
X = data[training_columns]
y = data['APOGEE_LOGG']  # You can change this to 'APOGEE_LOGG_SPEC' or 'APOGEE_RV_LOGG' if needed

# Create the preprocessing pipeline
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),  # Step 1: Impute missing values with mean
    ('scaling', StandardScaler()),  # Step 2: Standardize the features
    ('feature_selection', SelectKBest(score_func=f_regression, k=20)),  # Step 3: Select top 20 features
])

# Fit and transform the data
X_preprocessed = pipeline.fit_transform(X, y)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

# Initialize and train the model
rfr = RandomForestRegressor(n_estimators=100, random_state=42)
rfr.fit(X_train, y_train)

# Make predictions
y_pred = rfr.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)


In [5]:
# Create a subdirectory to save models and metrics
output_dir = 'RFRexp2'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save the trained model
model_path = os.path.join(output_dir, 'random_forest_regressor.joblib')
dump(rfr, model_path)

['RFRexp2/random_forest_regressor.joblib']

In [6]:
# Display metrics
print(f'Mean Squared Error: {mse}')
print(f'R² Score: {r2}')


# Save metrics as CSV
metrics_df = pd.DataFrame({
    'Mean Squared Error': [mse],
    'R² Score':[r2]
   
})
metrics_path = os.path.join(output_dir, 'metrics.csv')
metrics_df.to_csv(metrics_path, index=False)

Mean Squared Error: 0.1316941584507208
R² Score: 0.935822220600081


In [ ]:
from sklearn.model_selection import cross_val_score

# Perform 5-fold cross-validation
cv_scores = cross_val_score(rfr, X_preprocessed, y, cv=5, scoring='r2')
print(f'Cross-validated R² scores: {cv_scores}')
print(f'Mean Cross-validated R² score: {np.mean(cv_scores)}')



# Save cross-validation metrics to CSV
cv_metrics_path = os.path.join(output_dir, 'cv_metrics.csv')
cv_metrics_df.to_csv(cv_metrics_path, index=False)



In [ ]:
# Save cross-validation metrics to CSV
cv_metrics_df = pd.DataFrame({
    'Fold': np.arange(1, 6),
    'R² Score': cv_scores
})
cv_metrics_path = os.path.join(output_dir, 'cv_metrics.csv')
cv_metrics_df.to_csv(cv_metrics_path, index=False)


In [ ]:
import matplotlib.pyplot as plt

# Get feature importances
feature_importances = rfr.feature_importances_

# Get the names of the selected features
selected_features = pipeline.named_steps['feature_selection'].get_feature_names_out()

# Create a DataFrame for visualization
importances_df = pd.DataFrame({
    'Feature': selected_features,
    'Importance': feature_importances
})

# Sort by importance
importances_df = importances_df.sort_values(by='Importance', ascending=False)

# Plot feature importances
plt.figure(figsize=(10, 8))
plt.barh(importances_df['Feature'], importances_df['Importance'])
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Feature Importances')
plt.gca().invert_yaxis()
plt.show()


In [ ]:
# Save feature importances to CSV
importances_path = os.path.join(output_dir, 'feature_importances.csv')
importances_df.to_csv(importances_path, index=False)

# Save the plot
plot_path = os.path.join(output_dir, 'feature_importances.png')
plt.savefig(plot_path)